### Seq2Seq

In [11]:
import tensorflow as tf
import numpy as np
import time
import pickle
import operator

In [12]:
# data load
with open('para0529.txt', 'rb') as f:
    sentence = pickle.load(f)
    # sentence = sentence[0:20000]
with open('words0528.txt', 'rb') as f:   
    keyword = pickle.load(f)
    # keyword = keyword[0:20000]

In [13]:
## data set 만들기
new_keyword = []
for i in range(0,len(keyword)):
    a = " ".join(keyword[i])
    new_keyword.append(a)

corpus = []
for i in range(len(sentence)):
    corpus.append([sentence[i], new_keyword[i]])

In [14]:
# pickle에 저장하기
with open('corpus.txt', 'wb') as f:
    pickle.dump(corpus, f)
f.close

<function BufferedWriter.close>

In [15]:
## functions for preprocess
# import module
import numpy as np
import pandas as pd
from collections import defaultdict

# loading function          
def loading_data(data_path, eng=True, num=True, punc=False):
    with open('corpus.txt', 'rb') as f:
        corpus = pickle.load(f)
    
    corpus = np.array(corpus)
    title = []
    contents = []
    for doc in corpus:
        if type(doc[0]) is not str or type(doc[1]) is not str:
            title.append(doc[0])
            contents.append(doc[1])
    return title, contents

def make_dict_all_cut(contents, minlength, maxlength, jamo_delete=False):
    dict = defaultdict(lambda: [])
    for doc in contents:
        for idx, word in enumerate(doc.split(" ")):
            if len(word) > minlength:
                normalizedword = word[:maxlength]
                if jamo_delete:
                    tmp = []
                    for char in normalizedword:
                        if ord(char) < 10000 or ord(char) > 0:
                            tmp.append(char)
                    normalizedword = ''.join(char for char in tmp)
                if word not in dict[normalizedword]:
                    dict[normalizedword].append(word)
    dict = sorted(dict.items(), key=operator.itemgetter(0))[1:]
    words = []
    for i in range(len(dict)):
        word = []
        word.append(dict[i][0])
        for w in dict[i][1]:
            if w not in word:
                word.append(w)
        words.append(word)

    words.append(['<PAD>'])
    words.append(['<S>'])
    words.append(['<E>'])
    words.append(['<UNK>'])
    # word_to_ix, ix_to_word 생성
    ix_to_word = {i: ch[0] for i, ch in enumerate(words)}
    word_to_ix = {}
    for idx, words in enumerate(words):
        for word in words:
            word_to_ix[word] = idx
    return word_to_ix, ix_to_word

# making input function
def make_inputs(rawinputs, rawtargets, word_to_ix, encoder_size, decoder_size, shuffle=True):
    rawinputs = np.array(rawinputs)
    rawtargets = np.array(rawtargets)
    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(len(rawinputs)))
        rawinputs = rawinputs[shuffle_indices]
        rawtargets = rawtargets[shuffle_indices]
    encoder_input = []
    decoder_input = []
    targets = []
    target_weights = []
    for rawinput, rawtarget in zip(rawinputs, rawtargets):
        tmp_encoder_input = [word_to_ix[v] for idx, v in enumerate(rawinput.split()) if
                             idx < encoder_size and v in word_to_ix]
        encoder_padd_size = max(encoder_size - len(tmp_encoder_input), 0)
        encoder_padd = [word_to_ix['<PAD>']] * encoder_padd_size
        encoder_input.append(list(reversed(tmp_encoder_input + encoder_padd)))
        tmp_decoder_input = [word_to_ix[v] for idx, v in enumerate(rawtarget.split()) if
                             idx < decoder_size - 1 and v in word_to_ix]
        decoder_padd_size = decoder_size - len(tmp_decoder_input) - 1
        decoder_padd = [word_to_ix['<PAD>']] * decoder_padd_size
        decoder_input.append([word_to_ix['<S>']] + tmp_decoder_input + decoder_padd)
        targets.append(tmp_decoder_input + [word_to_ix['<E>']] + decoder_padd)
        tmp_targets_weight = np.ones(decoder_size, dtype=np.float32)
        tmp_targets_weight[-decoder_padd_size:] = 0
        target_weights.append(list(tmp_targets_weight))
    return encoder_input, decoder_input, targets, target_weights

# doclength check function
def check_doclength(docs, sep=True):
    max_document_length = 0
    for doc in docs:
        if sep:
            words = doc.split()
            document_length = len(words)
        else:
            document_length = len(doc)
        if document_length > max_document_length:
            max_document_length = document_length
    return max_document_length

# making batch function
def make_batch(encoder_inputs, decoder_inputs, targets, target_weights):
    encoder_size = len(encoder_inputs[0])
    decoder_size = len(decoder_inputs[0])
    encoder_inputs, decoder_inputs, targets, target_weights = np.array(encoder_inputs), np.array(decoder_inputs), np.array(targets), np.array(target_weights)
    result_encoder_inputs = []
    result_decoder_inputs = []
    result_targets = []
    result_target_weights = []
    for i in range(encoder_size):
        result_encoder_inputs.append(encoder_inputs[:, i])
    for j in range(decoder_size):
        result_decoder_inputs.append(decoder_inputs[:, j])
        result_targets.append(targets[:, j])
        result_target_weights.append(target_weights[:, j])
    return result_encoder_inputs, result_decoder_inputs, result_targets, result_target_weights

In [16]:
data_path = '../DJ'
model_path = '../saved_model_seq2seq'

In [17]:
# parameter
title, contents = loading_data(data_path, eng=False, num=False, punc=False)
word_to_ix, ix_to_word = make_dict_all_cut(title+contents, minlength=0, maxlength=100, jamo_delete=True)

# parameters
multi = True
forward_only = False
hidden_size = 200
vocab_size = len(ix_to_word)
num_layers = 3
learning_rate = 0.001
batch_size = 10
encoder_size = 100
decoder_size = check_doclength(title,sep=True) # (Maximum) number of time steps in this batch
steps_per_checkpoint = 10

# transform data
encoderinputs, decoderinputs, targets_, targetweights = make_inputs(contents, title, 
                                                                         word_to_ix,  
                                                                         encoder_size=encoder_size, 
                                                                         decoder_size=decoder_size, 
                                                                         shuffle=False)
print('컨텐츠 개수 :', len(contents))
print('단어 걔수 :', len(ix_to_word))

컨텐츠 개수 : 20000
단어 걔수 : 53782


In [21]:
# setting model
class seq2seq(object):

    def __init__(self, multi, hidden_size, num_layers, forward_only,
                 learning_rate, batch_size,
                 vocab_size, encoder_size, decoder_size):

        # variables
        self.source_vocab_size = vocab_size
        self.target_vocab_size = vocab_size
        self.batch_size = batch_size
        self.encoder_size = encoder_size
        self.decoder_size = decoder_size
        self.learning_rate = tf.Variable(float(learning_rate), trainable=False)
        self.global_step = tf.Variable(0, trainable=False)

        # networks
        W = tf.Variable(tf.random_normal([hidden_size, vocab_size]))
        b = tf.Variable(tf.random_normal([vocab_size]))
        output_projection = (W, b)
        self.encoder_inputs = [tf.placeholder(tf.int32, [batch_size]) for _ in range(encoder_size)]  # 인덱스만 있는 데이터 (원핫 인코딩 미시행)
        self.decoder_inputs = [tf.placeholder(tf.int32, [batch_size]) for _ in range(decoder_size)]
        self.targets = [tf.placeholder(tf.int32, [batch_size]) for _ in range(decoder_size)]
        self.target_weights = [tf.placeholder(tf.float32, [batch_size]) for _ in range(decoder_size)]

        # models
        if multi:
            single_cell = tf.nn.rnn_cell.GRUCell(num_units=hidden_size)
            cell = tf.nn.rnn_cell.MultiRNNCell([single_cell] * num_layers)
        else:
            cell = tf.nn.rnn_cell.GRUCell(num_units=hidden_size)
            #cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size)

        if not forward_only:
            self.outputs, self.states = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                self.encoder_inputs, self.decoder_inputs, cell,
                num_encoder_symbols=vocab_size,
                num_decoder_symbols=vocab_size,
                embedding_size=hidden_size,
                output_projection=output_projection,
                feed_previous=False)

            self.logits = [tf.matmul(output, output_projection[0]) + output_projection[1] for output in self.outputs]
            self.loss = []
            for logit, target, target_weight in zip(self.logits, self.targets, self.target_weights):
                crossentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=target)
                self.loss.append(crossentropy * target_weight)
            self.cost = tf.add_n(self.loss)
            self.train_op = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)


        else:
            self.outputs, self.states = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                self.encoder_inputs, self.decoder_inputs, cell,
                num_encoder_symbols=vocab_size,
                num_decoder_symbols=vocab_size,
                embedding_size=hidden_size,
                output_projection=output_projection,
                feed_previous=True)
            self.logits = [tf.matmul(output, output_projection[0]) + output_projection[1] for output in self.outputs]

    def step(self, session, encoderinputs, decoderinputs, targets, targetweights, forward_only):
        input_feed = {}
        for l in range(len(encoder_inputs)):
            input_feed[self.encoder_inputs[l].name] = encoderinputs[l]
        for l in range(len(decoder_inputs)):
            input_feed[self.decoder_inputs[l].name] = decoderinputs[l]
            input_feed[self.targets[l].name] = targets[l]
            input_feed[self.target_weights[l].name] = targetweights[l]
        if not forward_only:
            output_feed = [self.train_op, self.cost]
        else:
            output_feed = []
            for l in range(len(decoder_inputs)):
                output_feed.append(self.logits[l])
        output = session.run(output_feed, input_feed)
        if not forward_only:
            return output[1] # loss
        else:
            return output[0:] # outputs

In [ ]:
# trina
sess = tf.Session()
model = seq2seq(multi=multi, hidden_size=hidden_size, num_layers=num_layers,
                    learning_rate=learning_rate, batch_size=batch_size,
                    vocab_size=vocab_size,
                    encoder_size=encoder_size, decoder_size=decoder_size,
                    forward_only=forward_only)
sess.run(tf.global_variables_initializer())
step_time, loss = 0.0, 0.0
current_step = 0
start = 0
end = batch_size
while current_step < 1000000:

    if end > len(title):
        start = 0
        end = batch_size

    # Get a batch and make a step
    start_time = time.time()
    encoder_inputs, decoder_inputs, targets, target_weights = make_batch(encoderinputs[start:end],
                                                                        decoderinputs[start:end],
                                                                        targets_[start:end],
                                                                        targetweights[start:end])

    if current_step % steps_per_checkpoint == 0:
        for i in range(decoder_size - 2):
            decoder_inputs[i + 1] = np.array([word_to_ix['<PAD>']] * batch_size)
        output_logits = model.step(sess, encoder_inputs, decoder_inputs, targets, target_weights, True)
        predict = [np.argmax(logit, axis=1)[0] for logit in output_logits]
        predict = ' '.join(ix_to_word[ix] for ix in predict)
        real = [word[0] for word in targets]
        real = ' '.join(ix_to_word[ix] for ix in real)
        print('\n----\n step : %s \n time : %s \n LOSS : %s \n 예측 : %s \n 손질한 정답 : %s \n 정답 : %s \n ----' %
              (current_step, step_time, loss, predict, real, title[start]))
        loss, step_time = 0.0, 0.0
        
    step_loss = model.step(sess, encoder_inputs, decoder_inputs, targets, target_weights, False)
    step_time += time.time() - start_time / steps_per_checkpoint
    loss += np.mean(step_loss) / steps_per_checkpoint
    current_step += 1
    start += batch_size
    end += batch_size


----
 step : 0 
 time : 0.0 
 LOSS : 0.0 
 예측 : 한국인으로 건축공학 건축공학 건축공학 건축공학 건축공학 진단기기 진단기기 진단기기 진단기기 진단기기 진단기기 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 풀어보고자 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 여객운송 
 손질한 정답 : 중학 시절 은사님께서 해주신 말씀입니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 :  중학 시절 은사님께서 해주신 말씀입니다 
 ----

----
 step : 10 
 time : 13754649301.925253 
 LOSS : 158.79534149169925 
 예측 : 합리적 상황이었지만 승객 승객 헤아리고 헤아리고 헤아리고 헤아리고 헤아리고 사례는 사례는 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 3년 내에는 팀원들이 믿고 일을 맡길 수 있는 엔지니


----
 step : 130 
 time : 13754649690.770237 
 LOSS : 130.4274574279785 
 예측 : 그래서 그래서 면들은 면들은 면들은 수 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 그 이유는 팀원마다 번역에 차이가 있었기 때문입니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : 그 이유는 팀원마다 번역에 차이가 있었기 때문입니다 
 ----

----
 step : 140 
 time : 13754649723.281437 
 LOSS : 131.30414276123045 
 예측 : 그 어울려 선정에 선정에 선정에 선정에 선정에 대화제약의 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 외식경영관리사를 취득하여 프랜차이즈 사업에 대한 이론적 지식을 쌓아 가맹점주분들이 최상의 매출을 이룰 수 있게 도와드리겠습니다 <E> <P


----
 step : 260 
 time : 13754650113.468441 
 LOSS : 139.39704360961915 
 예측 : 저는 통해 통해 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : “도전을 위한 밑그림”저는 평소 계획을 짜는 것을 좋아합니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : “도전을 위한 밑그림”저는 평소 계획을 짜는 것을 좋아합니다 
 ----

----
 step : 270 
 time : 13754650146.092823 
 LOSS : 133.65087509155273 
 예측 : 이러한 이에 통해 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 앞으로는 이러한 패기를 가지고 새로운 것에 계속해서 도전하는 유니클로의 일원이 되겠습니다 <E> <PAD> <PAD> <PAD> <PA


----
 step : 390 
 time : 13754650536.735765 
 LOSS : 148.78917770385743 
 예측 : 그 생각합니다 생각합니다 위해 위해 위해 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 창업을 시작해 저희는 가장 먼저 재정적 어려움과 함께 첫 번째로 기획한 중개법이 적합하지 못해 창업이 좌초 위기에 놓였었습니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : 창업을 시작해 저희는 가장 먼저 재정적 어려움과 함께 첫 번째로 기획한 중개법이 적합하지 못해 창업이 좌초 위기에 놓였었습니다 
 ----

----
 step : 400 
 time : 13754650569.48361 
 LOSS : 143.09684295654296 
 예측 : 하지만 통해 통해 통해 통해 통해 통해 통해 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 질적 성장을 목표로 다양한 컨셉과 상품 운영을 통해 경쟁력 있는 GS25점포를 만들겠습니다 <E> <PAD> <PAD


----
 step : 520 
 time : 13754650960.246511 
 LOSS : 113.5031509399414 
 예측 : 저는 통해 통해 위해 위해 위해 위해 위해 위해 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 정체되지 않고 꾸준히 발전하고자 하는 저의 목표와 부합하는 고려해운과 함께 미래를 준비하고 싶습니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : 정체되지 않고 꾸준히 발전하고자 하는 저의 목표와 부합하는 고려해운과 함께 미래를 준비하고 싶습니다 
 ----

----
 step : 530 
 time : 13754650992.835358 
 LOSS : 156.15349884033202 
 예측 : 하지만 하지만 통해 통해 통해 통해 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 운송비를 절감하려면 어떤 것이 필요하겠냐는 질문에 기사 분께서는 연비 절감이라는 당연하지만 현장을 모르는 저희로써는 생각하지 

...


----
 step : 18550 
 time : 13754711657.187853 
 LOSS : 23.195943546295165 
 예측 : 저는 제가 제가 <E> <E> <E> 생각합니다 생각합니다 생각합니다 생각합니다 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 그리고 저는 우리라는 단어가 가지는 힘을 믿습니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : 그리고 저는 우리라는 단어가 가지는 힘을 믿습니다 
 ----

----
 step : 18560 
 time : 13754711691.045547 
 LOSS : 24.675390529632566 
 예측 : 이와 진심 진심 진심 역시 역시 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 이와 더불어 진심 역시 갖추고자 노력했습니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P


----
 step : 18680 
 time : 13754712152.886747 
 LOSS : 25.710698604583737 
 예측 : 이를 연구 연구 연구 연구 연구 기회를 기회를 기회를 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 이를 토대로 디스플레이 소재에 대한 연구 능력을 키울 수 있었습니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : 이를 토대로 디스플레이 소재에 대한 연구 능력을 키울 수 있었습니다 
 ----

----
 step : 18690 
 time : 13754712186.67635 
 LOSS : 19.6363618850708 
 예측 : 특히 OOOOOO대학교 범주가 범주가 범주가 범주가 범주가 부분이 부분이 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 특히 화재보험사의 판매 상품 범주가 넓어짐에 따라 보장 범위도 넓어져 경쟁력이 더욱 높아졌다고 생각합니다 <E> <PAD> <PAD>


----
 step : 18810 
 time : 13754712664.334112 
 LOSS : 16.58139891624451 
 예측 : 그러나 필요한 필요한 필요한 단점이 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 그러나 제가 필요하다고 결정한 이상 포기하고 싶지 않았습니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : 그러나 제가 필요하다고 결정한 이상 포기하고 싶지 않았습니다 
 ----

----
 step : 18820 
 time : 13754712717.617716 
 LOSS : 24.396297740936276 
 예측 : LS전선은 일생생활에서부터 일생생활에서부터 일생생활에서부터 이해를 경제 경제 경제 경제 현재 등을 등을 등을 등을 등을 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : LS전선은 일생생활에서부터 산업 부분까지 케이블 솔루션을 제공하고 뿐만 아니라 전 세계 시장을 바탕으로 전선


----
 step : 18940 
 time : 13754713208.383661 
 LOSS : 21.314942073822024 
 예측 : IS동서의 일원이 일원이 직무에 직무에 인재로 인재로 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : IS동서의 일원이 되어 미래를 선도하는 유능한 인재가 되겠습니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : IS동서의 일원이 되어 미래를 선도하는 유능한 인재가 되겠습니다 
 ----

----
 step : 18950 
 time : 13754713246.17522 
 LOSS : 28.403190994262697 
 예측 : 그래야 반품과 반품과 개인보다는 개인보다는 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 그래야 반품과 폐기의 양이 적어지기 때문입니다 <E> <PAD> <PAD> <PAD> <PAD> <PA


----
 step : 19070 
 time : 13754713798.36021 
 LOSS : 20.73372793197632 
 예측 : 하지만 별도의 별도의 비용이 비용이 할당되지 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : 하지만 별도의 마케팅 비용이 할당되지 않아 모두들 난감해하고 있었습니다 <E> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
 정답 : 하지만 별도의 마케팅 비용이 할당되지 않아 모두들 난감해하고 있었습니다 
 ----

----
 step : 19080 
 time : 13754713834.886787 
 LOSS : 16.433237075805668 
 예측 : 이로써 이미 이미 빠르게 빠르게 요구를 <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> <E> 
 손질한 정답 : LPG 산업은 앞으로 많은 변화와 성장의 계기가 있습니다 <E> <PAD> <PAD> <PAD> <PAD> <PA

In [20]:
tf.reset_default_graph()

In [83]:
contents[0]

'중학 시절 은사 말씀'